# Original Dataset - SVM

In [1]:
# Import dependencies
import sqlalchemy
from sqlalchemy import create_engine, inspect

import pandas as pd
import numpy as np
import os
import time
from datetime import datetime

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score

import joblib

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

In [3]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['alexaval',
 'alexbval',
 'alexgval',
 'alexrval',
 'aval',
 'bval',
 'chroma1',
 'chroma10',
 'chroma11',
 'chroma12',
 'chroma2',
 'chroma3',
 'chroma4',
 'chroma5',
 'chroma6',
 'chroma7',
 'chroma8',
 'chroma9',
 'chromastd',
 'deltachroma1',
 'deltachroma10',
 'deltachroma11',
 'deltachroma12',
 'deltachroma2',
 'deltachroma3',
 'deltachroma4',
 'deltachroma5',
 'deltachroma6',
 'deltachroma7',
 'deltachroma8',
 'deltachroma9',
 'deltachromastd',
 'deltaenergy',
 'deltaenergyentropy',
 'deltamfcc1',
 'deltamfcc10',
 'deltamfcc11',
 'deltamfcc12',
 'deltamfcc13',
 'deltamfcc2',
 'deltamfcc3',
 'deltamfcc4',
 'deltamfcc5',
 'deltamfcc6',
 'deltamfcc7',
 'deltamfcc8',
 'deltamfcc9',
 'deltaspectralcentroid',
 'deltaspectralentropy',
 'deltaspectralflux',
 'deltaspectralrolloff',
 'deltaspectralspread',
 'deltazcr',
 'demographic',
 'diagnosis',
 'energy',
 'energyentropy',
 'gval',
 'habits',
 'mfcc1',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5

In [4]:
# Tables to import
import_tables = ['diagnosis', 'demographic', 'habits']

In [5]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in import_tables:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [6]:
# Merge dataframes
merged_df = pd.merge(
    dataframes['demographic_df'],
    dataframes['diagnosis_df'],
    how = 'inner',
    on = 'id'
)

merged_df = pd.merge(
    merged_df,
    dataframes['habits_df'],
    how = 'inner',
    on = 'id'
)

# Display merged_df
merged_df.head()

,id,age,gender,occupation_status,diagnosis,subtype,vhi_score,rsi_score,reflux_indicated,vhi_zscore,...,chocolate,chocolate_grams_pd,coffee,coffee_pd,citrus_fruits,citrus_fruits_pd,soft_cheese,soft_cheese_pd,tomatoes,water_litres_pd
0,voice100,24,m,unknown,healthy,no subtype,0,5,0,-0.58,...,sometimes,30,always,3,never,0.00,almost always,100,never,1.5
1,voice101,60,m,unknown,healthy,no subtype,80,10,0,4.76,...,sometimes,30,always,4,never,0.00,sometimes,100,sometimes,1.5
2,voice192,22,m,cook,hyperkinetic dysphonia,no subtype,0,10,0,-0.58,...,always,14,always,3,almost always,1.17,sometimes,100,sometimes,2.5
3,voice193,46,f,housewife,hyperkinetic dysphonia,no subtype,0,36,1,-0.58,...,sometimes,30,always,2,sometimes,1.00,sometimes,100,sometimes,1.0
4,voice008,51,f,researcher,reflux laryngitis,no subtype,19,15,1,0.68,...,almost always,20,always,2,almost always,1.00,sometimes,100,almost always,1.0


## Preprocessing

### Separate the target and feature variables

In [7]:
# Drop the 'id' column
no_id_df = merged_df.drop(columns=['id'])

In [8]:
# Define the target variables
target_var = ['diagnosis', 'subtype']
y = no_id_df[target_var]

# Define the feature variables
X = no_id_df.drop(columns=target_var)

### Binary classification - `diagnosis`

In [9]:
# Encode the target variable, ignore subtype
y = y['diagnosis'].apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Bin `occupation_status` column

In [10]:
# Use limit_unique() function to bin the column
limit_unique(X, 10, ['occupation_status'])

occupation_status
unknown               41
researcher            41
other                 25
employee              25
housewife             23
student               16
technical operator    12
singer                10
pensioner              6
doctor                 5
Name: count, dtype: int64
Number of unique values: 10



### Encode feature columns

In [11]:
# Define the maps
smoker_map = {
    'no': 0,
    'casual': 1,
    'yes': 2
}

alcohol_map = {
    'nondrinker': 0,
    'casual': 1,
    'habitual': 2
}

habit_map = {
    'never': 0,
    'almost never': 1,
    'sometimes': 2,
    'almost always': 3,
    'always': 4
}

In [12]:
# Apply the label encoding using the maps
X['smoker'] = X['smoker'].map(smoker_map)
X['alcohol_consumption'] = X['alcohol_consumption'].map(alcohol_map)

# Habit columns
habit_cols = [
    'carbonated_beverages', 'tomatoes',
    'coffee', 'chocolate',
    'soft_cheese', 'citrus_fruits'
]

# Use a loop for the habit columns
for habit in habit_cols:
    X[habit] = X[habit].map(habit_map)

In [13]:
# Encode the categorical columns using get_dummies
categorical_hot = ['gender', 'occupation_status']

# One-hot encoding
encoded_columns = pd.get_dummies(X[categorical_hot]).astype(int)

# Update the feature dataframe
X.drop(categorical_hot, axis=1, inplace=True)
X = pd.concat([X, encoded_columns], axis=1)

# Convert the 'occupation_status_unknown' to 'occupation_status_other'
X.loc[X['occupation_status_unknown'] == 1, 'occupation_status_other'] = 1

# Drop the 'occupation_status_unknown' column
X = X.drop(columns=['occupation_status_unknown'])

In [14]:
# Display the dataframe
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   204 non-null    int64  
 1   vhi_score                             204 non-null    int64  
 2   rsi_score                             204 non-null    int64  
 3   reflux_indicated                      204 non-null    int64  
 4   vhi_zscore                            204 non-null    float64
 5   vhi_impact                            204 non-null    int64  
 6   alcohol_consumption                   204 non-null    int64  
 7   alcohol_pd                            204 non-null    float64
 8   smoker                                204 non-null    int64  
 9   cigarettes_pd                         204 non-null    int64  
 10  carbonated_beverages                  204 non-null    int64  
 11  carbonated_pd      

,age,vhi_score,rsi_score,reflux_indicated,vhi_zscore,vhi_impact,alcohol_consumption,alcohol_pd,smoker,cigarettes_pd,...,gender_m,occupation_status_doctor,occupation_status_employee,occupation_status_housewife,occupation_status_other,occupation_status_pensioner,occupation_status_researcher,occupation_status_singer,occupation_status_student,occupation_status_technical operator
0,24,0,5,0,-0.58,0,1,0.36,0,0,...,1,0,0,0,1,0,0,0,0,0
1,60,80,10,0,4.76,4,0,0.00,0,0,...,1,0,0,0,1,0,0,0,0,0
2,22,0,10,0,-0.58,0,0,0.00,0,0,...,1,0,0,0,1,0,0,0,0,0
3,46,0,36,1,-0.58,0,1,0.36,2,15,...,0,0,0,1,0,0,0,0,0,0
4,51,19,15,1,0.68,1,1,0.36,0,0,...,0,0,0,0,0,0,1,0,0,0


## Split and Scale

In [15]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Support Vector Machine

In [17]:
# Define the parameters
kernel = 'poly'

In [18]:
# Create an SVM instance
svm_model = SVC(kernel=kernel, class_weight='balanced')

# Fit the model
svm_model = svm_model.fit(X_train_scaled, y_train)

In [19]:
# Making predictions using the testing data
predictions = svm_model.predict(X_test_scaled)

In [20]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm,
    index = ["Actual 0", "Actual 1"],
    columns = ["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
model_accuracy = accuracy_score(y_test, predictions)

In [21]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"\nAccuracy Score: {model_accuracy}")
print("\nClassification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4,10
Actual 1,2,35



Accuracy Score: 0.7647058823529411

Classification Report
              precision    recall  f1-score   support

           0       0.67      0.29      0.40        14
           1       0.78      0.95      0.85        37

    accuracy                           0.76        51
   macro avg       0.72      0.62      0.63        51
weighted avg       0.75      0.76      0.73        51



In [22]:
# Extract the precision and recall
precision = precision_score(y_test, predictions, average=None)
recall = recall_score(y_test, predictions, average=None)
print(precision, recall)

[0.66666667 0.77777778] [0.28571429 0.94594595]


In [23]:
# Create dictionary to save the results
results_dict = dict()

current_time = datetime.now()
results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_accuracy'] = model_accuracy
results_dict['precision_0'] = precision[0]
results_dict['precision_1'] = precision[1]
results_dict['recall_0'] = recall[0]
results_dict['recall_1'] = recall[1]
results_dict['kernel'] = kernel

In [24]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  added class_weight = balanced


In [25]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 8, 21, 35, 22, 202016),
 'runtime': 0.11400485038757324,
 'model_accuracy': 0.7647058823529411,
 'precision_0': 0.6666666666666666,
 'precision_1': 0.7777777777777778,
 'recall_0': 0.2857142857142857,
 'recall_1': 0.9459459459459459,
 'kernel': 'poly',
 'change_message': 'added class_weight = balanced'}

In [26]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_accuracy,precision_0,precision_1,recall_0,recall_1,kernel,change_message
0,2024-01-08 21:35:22.202016,0.114005,0.764706,0.666667,0.777778,0.285714,0.945946,poly,added class_weight = balanced


In [27]:
# Performance tracker
tracker_path = "../resources/tracker/svm_performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    joblib.dump(svm_model, f'../models/svm/run_{len(tracker_df)}_{model_pct}.h5')

else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    joblib.dump(svm_model, f'../models/svm/run_0_{model_pct}.h5')

In [28]:
# Check the prediction's output probabilities
predicted_prob = svm_model.predict(X_test_scaled)
clean_prob = np.round(predicted_prob, 2)

# Round to the nearest integer and flatten
clean_predicted = np.round(predicted_prob).astype(int).flatten()

# Convert to a dataframe for readability
output_prob = pd.DataFrame({
    'Actual': y_test,
    'Predicted': clean_predicted,
    'Probability': clean_prob.flatten()
})

output_prob

,Actual,Predicted,Probability
120,1,1,1
32,1,1,1
136,0,0,0
160,1,1,1
192,1,1,1
124,1,1,1
131,1,0,0
108,1,1,1
97,0,0,0
75,1,1,1


In [29]:
incorrect_idx = output_prob.loc[output_prob['Actual'] != output_prob['Predicted']].index
incorrect_idx

Index([131, 83, 57, 21, 116, 54, 135, 111, 143, 1, 202, 134], dtype='int64')

In [30]:
dataframes['diagnosis_df'].iloc[incorrect_idx]

,id,diagnosis,subtype,vhi_score,rsi_score,reflux_indicated,vhi_zscore,vhi_impact
131,voice141,hypokinetic dysphonia,no subtype,26,6,0,1.15,2
83,voice107,healthy,no subtype,22,10,0,0.89,1
57,voice111,healthy,no subtype,0,21,1,-0.58,0
21,voice091,healthy,no subtype,0,14,1,-0.58,0
116,voice207,healthy,no subtype,0,26,1,-0.58,0
54,voice051,healthy,no subtype,2,13,1,-0.45,0
135,voice109,healthy,no subtype,11,14,1,0.15,1
111,voice121,hypokinetic dysphonia,laryngitis,31,4,0,1.49,2
143,voice099,healthy,no subtype,28,21,1,1.29,2
1,voice101,healthy,no subtype,80,10,0,4.76,4
